### Applied Data Science Capstone - Week 3 Assignment, Segmenting and Clustering Neighborhoods in Toronto

*by Chufan Wu*

*Feb.2021*


**the content under this line is for task 1**

---
#### 1. import the numpy and pandas library

#### 2. create a data frame called df which contains the data of neighborhoods in Toronto

#### 3. use df.shape to display rows and columns of the data frame

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup


source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html.parser')

list_of_lists = []
for child in soup.find_all('table')[0].children:
    for td in child:
        if td == '\n': continue
        list_of_lists.append(td.text.strip().split('\n\n'))


In [2]:
df = pd.DataFrame({list_of_lists[0][0]:[list_of_lists[i+1][0] for i in range(len(list_of_lists)-1)], 
                  list_of_lists[0][1]:[list_of_lists[i+1][1] for i in range(len(list_of_lists)-1)],
                  list_of_lists[0][2]:[list_of_lists[i+1][2] for i in range(len(list_of_lists)-1)]})
df = df[df.Borough != 'Not assigned']
df.reset_index(inplace = True)
df = df.drop(['index'], axis = 1)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


*Data Source: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M*

In [3]:
df.shape

(103, 3)

**the content under this line is for task 2**

---

#### 1. read the provided CSV file to coor_df

#### 2. merge two data frames on condition Postal Code, gain requested data frame df

In [4]:
coor_df = pd.read_csv('http://cocl.us/Geospatial_data')
coor_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [5]:
df = pd.merge(df, coor_df, how = 'inner', left_on = 'Postal Code', right_on = 'Postal Code')
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


**the content under this line is for task 3**

---

#### 1. set up for Foursquare queries

In [6]:
import requests # library to handle requests
import random # library for random number generation

#!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

CLIENT_ID = 'EQJVOJSM4ZQWJN41MPFRQXXTIBDZBK0LDTC3XKEKV5ZUJPUN' # your Foursquare ID
CLIENT_SECRET = 'CVEALTAZKFOTKRFMWEWGAPSU3KXRTAFSS2D4PGFH2EOUW31N' # your Foursquare Secret
ACCESS_TOKEN = 'ZOEZHKAS2QXMS1RFAMYFG4Z5QX51ZHF3O0K3E13OO4NXDMEV' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

user_id = '484542633' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, VERSION) # define URL

# send GET request
results = requests.get(url).json()



Folium installed
Libraries imported.


#### 2. I'd like to check Japanese restaurants around Downtown Toronto Borough, so I'm going to set up the query

In [7]:
address = 'Downtown Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

search_query = 'Japanese'
radius = 500
print(search_query + ' .... OK!')

43.6541737 -79.38081164513409
Japanese .... OK!


In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=EQJVOJSM4ZQWJN41MPFRQXXTIBDZBK0LDTC3XKEKV5ZUJPUN&client_secret=CVEALTAZKFOTKRFMWEWGAPSU3KXRTAFSS2D4PGFH2EOUW31N&ll=43.6541737,-79.38081164513409&oauth_token=ZOEZHKAS2QXMS1RFAMYFG4Z5QX51ZHF3O0K3E13OO4NXDMEV&v=20180604&query=Japanese&radius=500&limit=30'

#### 3. handle the json result and load it into a pandas data frame

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '602687f4c005d56f944e9487'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '5d782433f6e3190008b1a914',
    'name': 'Gyubee Japanese Grill',
    'location': {'address': '157 Dundas St W',
     'lat': 43.655363,
     'lng': -79.384955,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.655363,
       'lng': -79.384955}],
     'distance': 359,
     'postalCode': 'M5B 1E4',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['157 Dundas St W', 'Toronto ON M5B 1E4', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d111941735',
      'name': 'Japanese Restaurant',
      'pluralName': 'Japanese Restaurants',
      'shortName': 'Japanese',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/japanese_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1613137908',
    'hasPerk': Fa

In [10]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,5d782433f6e3190008b1a914,Gyubee Japanese Grill,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1613137908,False,157 Dundas St W,43.655363,-79.384955,"[{'label': 'display', 'lat': 43.655363, 'lng':...",359,M5B 1E4,CA,Toronto,ON,Canada,"[157 Dundas St W, Toronto ON M5B 1E4, Canada]",NaN,NaN
1,56dd9d68498eb4e5edcb30f9,Spring Rolls | Japanese Restaurant in Toronto,"[{'id': '56aa371be4b08b9a8d573538', 'name': 'T...",v-1613137908,False,"40 Dundas St W Box 36,",43.656105,-79.383495,"[{'label': 'display', 'lat': 43.65610515821675...",304,M5G 2C2,CA,Toronto,ON,Canada,"[40 Dundas St W Box 36,, Toronto ON M5G 2C2, C...",NaN,NaN
2,4fbbb3ace4b01e39077b4d55,Akashiro Japanese Restaurant & Bar,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1613137908,False,220 Yonge St.,43.655965,-79.380541,"[{'label': 'display', 'lat': 43.65596504293232...",200,NaN,CA,Toronto,ON,Canada,[220 Yonge St. (in Toronto Eaton Centre Urban ...,in Toronto Eaton Centre Urban Eatery,NaN
3,4ae4b055f964a520229d21e3,Kyoto House Japanese Restaurant,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1613137908,False,143 Dundas St. West,43.655381,-79.385270,"[{'label': 'display', 'lat': 43.65538110598594...",383,NaN,CA,Toronto,ON,Canada,"[143 Dundas St. West, Toronto ON, Canada]",NaN,NaN
4,4c12b8ba82a3c9b6083ffaf8,Mt. Fuji Japanese Restaurant,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1613137908,False,NaN,43.658555,-79.384992,"[{'label': 'display', 'lat': 43.658555, 'lng':...",592,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN


In [11]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Gyubee Japanese Grill,Japanese Restaurant,157 Dundas St W,43.655363,-79.384955,"[{'label': 'display', 'lat': 43.655363, 'lng':...",359,M5B 1E4,CA,Toronto,ON,Canada,"[157 Dundas St W, Toronto ON M5B 1E4, Canada]",NaN,NaN,5d782433f6e3190008b1a914
1,Spring Rolls | Japanese Restaurant in Toronto,Theme Restaurant,"40 Dundas St W Box 36,",43.656105,-79.383495,"[{'label': 'display', 'lat': 43.65610515821675...",304,M5G 2C2,CA,Toronto,ON,Canada,"[40 Dundas St W Box 36,, Toronto ON M5G 2C2, C...",NaN,NaN,56dd9d68498eb4e5edcb30f9
2,Akashiro Japanese Restaurant & Bar,Sushi Restaurant,220 Yonge St.,43.655965,-79.380541,"[{'label': 'display', 'lat': 43.65596504293232...",200,NaN,CA,Toronto,ON,Canada,[220 Yonge St. (in Toronto Eaton Centre Urban ...,in Toronto Eaton Centre Urban Eatery,NaN,4fbbb3ace4b01e39077b4d55
3,Kyoto House Japanese Restaurant,Sushi Restaurant,143 Dundas St. West,43.655381,-79.385270,"[{'label': 'display', 'lat': 43.65538110598594...",383,NaN,CA,Toronto,ON,Canada,"[143 Dundas St. West, Toronto ON, Canada]",NaN,NaN,4ae4b055f964a520229d21e3
4,Mt. Fuji Japanese Restaurant,Japanese Restaurant,NaN,43.658555,-79.384992,"[{'label': 'display', 'lat': 43.658555, 'lng':...",592,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,4c12b8ba82a3c9b6083ffaf8
5,Gyu-Kaku Japanese BBQ,Japanese Restaurant,81 Church St,43.651422,-79.375047,"[{'label': 'display', 'lat': 43.65142227549791...",556,M5C 2G2,CA,Toronto,ON,Canada,"[81 Church St (at Adelaide St E), Toronto ON M...",at Adelaide St E,NaN,574ad72238fa943556d93b8e
6,KAKA,Japanese Restaurant,655 Bay Street,43.657457,-79.384192,"[{'label': 'display', 'lat': 43.65745745164475...",455,M5G 1Z4,CA,Toronto,ON,Canada,"[655 Bay Street (( Bay st & Elm st)), Toronto ...",( Bay st & Elm st),NaN,5a62412b1108ba191da5ba06
7,Uncle Tetsu's Japanese Cheesecake,Dessert Shop,598 Bay St,43.656063,-79.383695,"[{'label': 'display', 'lat': 43.65606287245643...",313,M5G 1M5,CA,Toronto,ON,Canada,"[598 Bay St (at Dundas St), Toronto ON M5G 1M5...",at Dundas St,NaN,55074b57498ef35eeeaafe29
8,Ninki Sushi,Japanese Restaurant,40 King St W,43.649812,-79.379518,"[{'label': 'display', 'lat': 43.64981217424400...",496,M5H 1A1,CA,Toronto,ON,Canada,"[40 King St W (at Bay St), Toronto ON M5H 1A1,...",at Bay St,Financial District,4beb2c0ba9900f475b7a1740
9,Kenzo Ramen,Ramen Restaurant,138 Dundas St. W,43.655540,-79.384866,"[{'label': 'display', 'lat': 43.65554007474909...",360,M5G 1C3,CA,Toronto,ON,Canada,"[138 Dundas St. W (at Elizabeth St.), Toronto ...",at Elizabeth St.,NaN,4ae354f7f964a520919321e3


In [12]:
dataframe_filtered.name

0                            Gyubee Japanese Grill
1    Spring Rolls | Japanese Restaurant in Toronto
2               Akashiro Japanese Restaurant & Bar
3                  Kyoto House Japanese Restaurant
4                     Mt. Fuji Japanese Restaurant
5                            Gyu-Kaku Japanese BBQ
6                                             KAKA
7                Uncle Tetsu's Japanese Cheesecake
8                                      Ninki Sushi
9                                      Kenzo Ramen
Name: name, dtype: object

#### 4. Now we draw a map with markers of Japanese restaurants using Foursquare APIs

In [13]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Downtown Toronto

# add a red circle marker to represent the Downtown Toronto
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Japanese restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map